In [1]:
# Importación de Librerías
import numpy as np
import pandas as pd
from scipy import io, integrate, linalg, signal
from scipy.sparse.linalg import eigs
import math
import matplotlib
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio

In [2]:
#pio.templates.default = "ggplot2"

In [3]:
# Definición de la función para potenciales cuadrados
# Modelo de Kronig-Penney
def Periodic_Potential(N,V_0,p_1,p_2):
  #Generación primaria de los enteros
  jj = np.arange(N+1)+1 
  # Re-escritura de los enteros
  # Método descrito en el paper
  a = (1+(2*jj-1)*(-1)**jj)/4 

  # Generación de combinaciones con meshgrid
  p, q = np.meshgrid(a,a)

  n = np.concatenate((q.reshape(len(a)**2, 1),p.reshape(len(a)**2, 1)),axis = 1)

  nx = n[:,0]
  ny = n[:,1]

  # Incremento de codificar energía
  nSqr = np.zeros(((N+1)*(N+1),1))
  n = np.concatenate((nSqr,n),axis = 1)

  n[:,0] = n[:,1]*n[:,1] + n[:,2]*n[:,2];

  index = np.argsort(n[:,0], kind="mergesort")

  n = n[index,:]
  nx = n[:,1]
  ny = n[:,2]
  n2 = n[:,0]
  # Definición de la diagonal del Hamiltoniano
  p = p_2 - p_1
  h = np.diag(4*n2+V_0*p*p)

  # Precálculo de los elementos fuera de la diagonal
  No_diag = np.zeros((2*N+1))
  for ii in range(2*N+1):
    if ii == N:
      No_diag[ii] = np.nan
    else:
      No_diag[ii] = 1/(2*np.pi*1j*(ii-N))*(-np.exp(1j*2*np.pi*(ii-N)*p_1) + np.exp(1j*2*np.pi*(ii-N)*p_2))
  # Cálculo de los elementos fuera de la diagonal
  II = 0
  JJ = 0
  for kk in range((N+1)*(N+1)):
    for ll in range(kk+1,(N+1)**2):
      if nx[kk] == nx[ll]:
        II = p;
      else:
        II = No_diag[int(nx[kk]-nx[ll])+N+1-1]

      if ny[kk] == ny[ll]:
        JJ = p;
      else:
        JJ = No_diag[int(ny[kk]-ny[ll])+N+1-1]
      h[kk,ll] = h[kk,ll] + V_0*II*JJ
  h = h + np.transpose(np.triu(h,k=1))
  
  return h

In [4]:
def muffinPotential(N,V_0,p_1,p_2):
  #Generación primaria de los enteros
  jj = np.arange(N+1)+1 
  # Re-escritura de los enteros
  # Método descrito en el paper
  a = (1+(2*jj-1)*(-1)**jj)/4 

  # Generación de combinaciones con meshgrid
  p, q = np.meshgrid(a,a)

  n = np.concatenate((q.reshape(len(a)**2, 1),p.reshape(len(a)**2, 1)),axis = 1)

  nx = n[:,0]
  ny = n[:,1]

  # Incremento de codificar energía
  nSqr = np.zeros(((N+1)*(N+1),1))
  n = np.concatenate((nSqr,n),axis = 1)

  n[:,0] = n[:,1]*n[:,1] + n[:,2]*n[:,2];

  index = np.argsort(n[:,0], kind="mergesort")

  n = n[index,:]
  nx = n[:,1]
  ny = n[:,2]
  n2 = n[:,0]

  # Definición de la diagonal del Hamiltoniano
  p = p_2 - p_1
  h = np.diag(4*n2+V_0*p*p)

  # Número de puntos para el rango de integración
  pointsInt = 100
  L = 1
  # Definición de los vectores para integrar
  x = np.linspace(0,L,pointsInt)
  y = x.copy()
  # Mesh para zona de integración
  X, Y = np.meshgrid(x,y)
  area = np.zeros((pointsInt,pointsInt))
  # Radio del círculo de la zona de integración
  R = p/2

  # En el círculo de integración, se colocan 1s,
  # fuera del círculo se quedan los 0s
  for ii in range(len(x)):
    for jj in range(len(y)):
      r = np.sqrt((x[ii]-L/2)**2+(y[jj]-L/2)**2)
      if r < R:
        area[ii,jj] = 1

  #fig = go.Figure(data=[
  #  go.Surface(z=area,x = x, y=y),
  #  ])
  #fig.show()
  # Cálculo de los elementos del hamiltoniano
  for kk in range((N+1)*(N+1)):
    for ll in range(kk+1,(N+1)**2):
      # Función a integrar
      F = np.exp(1j*2*np.pi*(nx[kk]-nx[ll])*X)*np.exp(1j*2*np.pi*(ny[kk]-ny[ll])*Y)
      # Convertir los valores fuera de la región a 0
      F = np.real(F)*area
      # Integración
      element = np.trapz(np.trapz(F, y, axis=0), x, axis=0)
      h[kk,ll] = element*V_0
  # Relleno de la matriz triangular
  h = h + np.transpose(np.triu(h,k=1))
  return h

In [5]:
# Definición de constantes importantes
Nest = 5 # Npumero de estados
v0 = -1 #Profundidad del pozo
P1 = 0.25 #Inicio del pozo
P2 = 0.75 #Final del pozo
repetition = 100 #Número de iteraciones

In [6]:
E = np.zeros((repetition+1,repetition+1,(Nest+1)**2)) #Inicialización Energías

# Se descomenta la función a usar según el modelo deseado,
# Kronig-Penney o Muffin:

#dimLessHam = Periodic_Potential(Nest,v0,P1,P2)
dimLessHam = muffinPotential(Nest,v0,P1,P2) 

In [7]:
jj = np.arange(Nest+1)+1

# Re-escritura de los enteros
# Método descrito en el paper
a = (1+(2*jj-1)*(-1)**jj)/4

# Generación de combinaciones con meshgrid
p, q = np.meshgrid(a,a)

n = np.concatenate((q.reshape(len(a)**2, 1),p.reshape(len(a)**2, 1)),axis = 1)

nx = n[:,0]
ny = n[:,1]

# Incremento de codificar energía
nSqr = np.zeros(((Nest+1)*(Nest+1),1))
n = np.concatenate((nSqr,n),axis = 1)

n[:,0] = n[:,1]*n[:,1] + n[:,2]*n[:,2];

index = np.argsort(n[:,0], kind="mergesort")

n = n[index,:]
nx = n[:,1]
ny = n[:,2]
n2 = n[:,0]

In [8]:
# Definición de los vectores kx, ky
kx = np.arange(0,np.pi+np.pi/repetition,np.pi/repetition)
ky = kx.copy()

# Paŕametro para indexar la matriz
m = np.arange((Nest+1)**2)

In [9]:
# Cálculo de los elementos de Bloch
# Para metrizando de Gamma a X'
for ii in range(repetition+1):
  bloch = (4/np.pi)*(nx[m]*kx[0]+ny[m]*ky[ii])+(kx[0]*kx[0]+ky[ii]*ky[ii])/(np.pi**2)  
   
  eigValues, eigVectors = np.linalg.eig(dimLessHam+np.diag(bloch,k = 0))
  eig_or = np.sort(eigValues, kind="mergesort")
  E[0,ii,:] = eig_or

In [10]:
# Cálculo de los elementos de Bloch
# Para metrizando de X' a M
for ii in range(repetition+1):
  bloch = (4/np.pi)*(nx[m]*kx[ii]+ny[m]*ky[repetition])+(kx[ii]*kx[ii]+ky[repetition]*ky[repetition])/(np.pi**2)
  eigValues, eigVectors = np.linalg.eig(dimLessHam+np.diag(bloch,k = 0))
  eig_or = np.sort(eigValues, kind="mergesort")
  E[ii,repetition,:] = eig_or

  

In [11]:
# Cálculo de los elementos de Bloch
# Para metrizando de M a Gamma
for ii in range(repetition,-1,-1):
  bloch = (4/np.pi)*(nx[m]*kx[ii]+ny[m]*ky[ii])+(kx[ii]*kx[ii]+ky[ii]*ky[ii])/(np.pi**2)
  eigValues, eigVectors = np.linalg.eig(dimLessHam+np.diag(bloch,k = 0))
  eig_or = np.sort(eigValues, kind="mergesort")
  E[ii,ii,:] = eig_or

In [12]:
# Energías para graficar
En2Plot = np.zeros((3*repetition,(Nest+1)**2))
K = np.arange(0,1+1/(3*repetition),1/(3*repetition))

# Se agrega un término de escala (sqrt(2))
K1 = K[0:2*repetition]
K2 = K[2*repetition+1:]
K = np.concatenate((K1,K2+np.sqrt(2)*(1/3)*(K2-(2/3))),axis = 0)

In [13]:
# Llenado de las energías para graficar
for ii in range(repetition):
  En2Plot[ii,:] = E[0,ii,:]
  En2Plot[ii+repetition,:] = E[ii,repetition,:]
  En2Plot[ii+2*repetition,:] = E[repetition-ii,repetition-ii,:] #Dudas


In [14]:
# Grafica de las bandas de energía
fig = px.line(x=K, y=En2Plot[:,0], labels={'x': "", 'y':'$E/E_{ISW}$'}) # override keyword names with labels
for i in range(8):  
  fig.add_trace(go.Scatter(mode = "lines", x = K, y = En2Plot[:,i+1]))
  fig.update_layout(width = 1000, height = 800)
  fig.update_traces(line = dict(width = 3.5))
  fig.update_yaxes(range = [-1,5])
  fig.update_xaxes(range = [0,1.1571])
fig.show()